In [1]:
import numpy as np
import stim

In [2]:
def noisy_bell_pair(num_pairs: int, x_err_prob: float, y_err_prob: float, z_err_prob: float) -> stim.Circuit:
    if x_err_prob + y_err_prob + z_err_prob > 1:
        raise ValueError(f"total error cannot exceed 1 : sum of args equal {x_err_prob + y_err_prob + z_err_prob}")
    circuit = stim.Circuit()
    circuit.append("H", [i for i in range(0, 2 * num_pairs, 2)])
    circuit.append("CNOT", [i for i in range(2 * num_pairs)])
    circuit.append(f"PAULI_CHANNEL_1", [i for i in range(0, 2 * num_pairs, 2)], (x_err_prob, y_err_prob, z_err_prob))
    return circuit

In [3]:
def process_bell_pair_statistics(sample_arrays: np.ndarray):
    # we assume that the last two measurements are for x_error and z_error
    # and that all measurements before the last two must all agree 
    no_err_count, x_err_count, y_err_count, z_err_count = 0, 0, 0, 0
    n = len(sample_arrays[0])
    m = (n - 2) // 2
    for meas in sample_arrays:
        # check meas[0:m+1] == meas[m:m+m+1]
        for i in range((n-2) // 2):
            if meas[i] != meas[i + m]:
                break
        else:
            z_error, x_error = meas[-2], meas[-1]
            if x_error and z_error:
                y_err_count += 1
            elif x_error:
                x_err_count += 1
            elif z_error:
                z_err_count +=1
            else:
                no_err_count += 1
    
    return (no_err_count, x_err_count, y_err_count, z_err_count), (no_err_count + x_err_count + y_err_count + z_err_count) / len(sample_arrays)

In [4]:
def pretty_print_result(protocol_name: str, x_err_prob: float, y_err_prob: float, z_err_prob: float, numshot: int, result_tuple: tuple[tuple[int, int, int, int], float]) -> None:
    longest_len = max([len(str(i)) for i in result_tuple[0]])
    print("==========================================")
    print(f"simulating {protocol_name} with {numshot}")
    print(f"    initial error probability")
    print(f"        I: {(1 - x_err_prob - y_err_prob - z_err_prob):.5f}")
    print(f"        X: {(x_err_prob):.5f}")
    print(f"        Y: {(y_err_prob):.5f}")
    print(f"        Z: {(z_err_prob):.5f}")
    print(f"    Output (fidelity: {result_tuple[0][0] / np.sum(result_tuple[0])}):")
    print(f"        success probability: {result_tuple[1]:.5f} ({np.sum(result_tuple[0])}/{numshot})")
    print(f"        I: {result_tuple[0][0] / (np.sum(result_tuple[0])):.5f} ({result_tuple[0][0]:>{longest_len}}/{np.sum(result_tuple[0])})")
    print(f"        X: {result_tuple[0][1] / (np.sum(result_tuple[0])):.5f} ({result_tuple[0][1]:>{longest_len}}/{np.sum(result_tuple[0])})")
    print(f"        Y: {result_tuple[0][2] / (np.sum(result_tuple[0])):.5f} ({result_tuple[0][2]:>{longest_len}}/{np.sum(result_tuple[0])})")
    print(f"        Z: {result_tuple[0][3] / (np.sum(result_tuple[0])):.5f} ({result_tuple[0][3]:>{longest_len}}/{np.sum(result_tuple[0])})")

In [5]:
def zz_detection_purification(x_err_prob: float, y_err_prob: float, z_err_prob: float):
    numshots = 10_000_000

    circuit = noisy_bell_pair(2, x_err_prob, y_err_prob, z_err_prob)
    circuit.append("CNOT", [0, 2, 1, 3])
    circuit.append("M", [2, 3])
    circuit.append("CNOT", [0, 1]) # rec(0) rec(1)
    circuit.append("H", 0)
    circuit.append("M", [0, 1]) # rec(2) rec(3)
    
    sampler = circuit.compile_sampler()
    result = process_bell_pair_statistics(sampler.sample(shots=numshots))
    pretty_print_result("Single Selection Detecting X/Y Error (ZZ syndrome) Purification", x_err_prob, y_err_prob, z_err_prob, numshots, result)

def xx_detection_purification(x_err_prob: float, y_err_prob: float, z_err_prob: float):
    numshots = 10_000_000

    circuit = noisy_bell_pair(2, x_err_prob, y_err_prob, z_err_prob)
    circuit.append("CNOT", [2, 0, 3, 1])
    circuit.append("MX", [2, 3])
    circuit.append("CNOT", [0, 1]) # rec(0) rec(1)
    circuit.append("H", 0)
    circuit.append("M", [0, 1]) # rec(2) rec(3)
    
    sampler = circuit.compile_sampler()
    result = process_bell_pair_statistics(sampler.sample(shots=numshots))
    pretty_print_result("Single Selection Detecting Y/Z Error (XX syndrome) Purification", x_err_prob, y_err_prob, z_err_prob, numshots, result)

def yy_detection_purification(x_err_prob: float, y_err_prob: float, z_err_prob: float):
    numshots = 10_000_000

    circuit = noisy_bell_pair(2, x_err_prob, y_err_prob, z_err_prob)
    circuit.append("S_DAG", [0, 1, 2, 3])
    circuit.append("CNOT", [2, 0, 3, 1])
    circuit.append("MX", [2, 3])
    circuit.append("S", [0, 1])
    circuit.append("CNOT", [0, 1]) # rec(0) rec(1)
    circuit.append("H", 0)
    circuit.append("M", [0, 1]) # rec(2) rec(3)
    
    sampler = circuit.compile_sampler()
    result = process_bell_pair_statistics(sampler.sample(shots=numshots))
    pretty_print_result("Single Selection Detecting X/Z Error (YY syndrome) Purification", x_err_prob, y_err_prob, z_err_prob, numshots, result)

In [6]:
# for Werner state, we expect all to be equal (close to each other)
zz_detection_purification(0.05, 0.05, 0.05)
xx_detection_purification(0.05, 0.05, 0.05)
yy_detection_purification(0.05, 0.05, 0.05)

simulating Single Selection Detecting X/Y Error (ZZ syndrome) Purification with 10000000
    initial error probability
        I: 0.85000
        X: 0.05000
        Y: 0.05000
        Z: 0.05000
    Output (fidelity: 0.8839229602981231):
        success probability: 0.82012 (8201174/10000000)
        I: 0.88392 (7249206/8201174)
        X: 0.00609 (  49919/8201174)
        Y: 0.00609 (  49978/8201174)
        Z: 0.10390 ( 852071/8201174)
simulating Single Selection Detecting Y/Z Error (XX syndrome) Purification with 10000000
    initial error probability
        I: 0.85000
        X: 0.05000
        Y: 0.05000
        Z: 0.05000
    Output (fidelity: 0.8840376660234543):
        success probability: 0.82010 (8201025/10000000)
        I: 0.88404 (7250015/8201025)
        X: 0.10370 ( 850435/8201025)
        Y: 0.00611 (  50125/8201025)
        Z: 0.00615 (  50450/8201025)
simulating Single Selection Detecting X/Z Error (YY syndrome) Purification with 10000000
    initial error probabili